<a href="https://colab.research.google.com/github/Peantoo/DE-Notebooks/blob/main/DE-Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

np.random.seed(0) 

# Data
## Read in CSV file
This is on my Google drive, but it can be brought in from anywhere. I'm bringing in v5 from the dataset. 

There are many ways to read in the data, but this is easiest for me as it lets me quickly access the data with Google Colab from any PC.
 
I always name the first CSV "RAW" so that it can be referenced later for comparison.

In [ ]:
RAW = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Sets/NFL Play by Play 2009-2018 (v5).csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
## First inspection of the data to get a feel for what I'm working with. 

RAW.describe()


In [ ]:
#That's a lot of data. First thing I notice is that some columns have zero in count, meaning there's no data. There's also a lot of columns missing. I want to see them all.

RAW.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449371 entries, 0 to 449370
Data columns (total 255 columns):
 #    Column                                Dtype  
---   ------                                -----  
 0    play_id                               int64  
 1    game_id                               int64  
 2    home_team                             object 
 3    away_team                             object 
 4    posteam                               object 
 5    posteam_type                          object 
 6    defteam                               object 
 7    side_of_field                         object 
 8    yardline_100                          float64
 9    game_date                             object 
 10   quarter_seconds_remaining             float64
 11   half_seconds_remaining                float64
 12   game_seconds_remaining                float64
 13   game_half                             object 
 14   quarter_end                           int64  
 15 

In [ ]:
#Make a dataframe from a copy of the RAW file to begin actual work on. Keep df_RAW so I can compare the dataframes later.

df_RAW = pd.DataFrame(RAW.copy())
df = df_RAW.copy()

In [ ]:
df.shape # Will need to clean up both columns and rows. Start with Columns as that will yield more cuts than Rows.

(449371, 255)

In [ ]:
#To see all of the rows, we much change the number of rows displayed.

pd.set_option("display.max_rows", 300)


In [ ]:
#I'd like to see what it actually looks like as a dataframe. It's obvious that some have 0 and some have NaN. It's a good idea to find out why and what correlation exists among columns. 
df.sample(20)

#Exploration & Assessment of missing data points and values
How many entries are NaN in the dataframe? How many entries are NaN in each column? This will tell me if any columns can be dropped. I know I can just immediately drop all columns and rows that are entirelly NaN, but I like to see the data first before erasing anything. 

df.isna().sum(axis=0) will display the number of NaN in each column in a new set of rows.

I combined the two topics of Exploration and Assessment as I tend to do both at the same time. 

In [ ]:
#How many NaN total?
num_nan = df.isna().sum().sum()
print(num_nan)

44121484


In [ ]:
#Percentage of NaN to Real?
total = (df.isna().sum().sum())+(df.count().sum().sum())
ratio = ((num_nan)/(total))*100
print(ratio)

38.50391490571942


In [ ]:
##Display Total Number of NaN in each column
df.isna().sum(axis=0)

A quick analysis tells me that something isn't quite right, as the number 12874 shows up a lot, particularly in areas where the data should be common to every game, like "Touchdowns" and "Pass attempt." These are areas where there should almost always be a value. I don't think I've seen a game where they never tried to pass. This tells me that the recorded games that have a null value in those areas probably need to be culled, as they will not contain any useful data. I would double check the CSV for those games, then delete those 12874 games.

I also noticed that the percentage of NaN in the whole set is oddly high. This is probably due to a large number of values that should be recorded as 0, not NaN. The columns with high amounts of null values for rarer things like "penalty yards," should not be culled, as these are rarer things that could influence the game. This shows that a high number of null values isn't necessarily a bad thing and you shouldn't always remove categories with high NaN numbers. For these, I would replace NaN with zero. 

In [ ]:
## Check for Columns where every value is NaN
df[df.columns[df.isna().any()]].isna().sum() * 100 / df.shape[0]
# Use this to find the percentage of the column that's NaN. Useful for getting rid of columns with no data or
#deciding which columns to fill with 0's or synthetic data. Does not include columns with no NaN data.

Any column where the value is 100 can safely be cut as it contains no information. I also now feel confident in deleting all rows where every column is NaN and vice-versa. 

#Drop missing values

Dropping values is never what I want to do, but sometimes it's necessary. I always start with getting rid of any columns with no data. I then move on to rows. I then check to see if any important data is missing and whether that row can be salvaged. After that, I then check to see the methods for filling missing data. This is probably the most time consuming part of the process, but it can also make the biggest difference if you're wanting high fidelity data sets.

In [ ]:
#Cut Columns and Rows with NaN for all values. I expect that if I remove any rows where all columns are NaN, then I will clear out the 12874 games from before.
df = df.dropna(axis=1, how='all')  
df = df.dropna(axis=0, how='all')

In [ ]:
a = pd.Series(df.shape)
a

In [ ]:
#Check 
df[df.columns[df.isnull().any()]].isnull().sum() * 100 / df.shape[0]

I noticed that this didn't get rid of all the columns with 2.86% NaN (the 12874 rows previously mention.) I need to do some further assessment of one of them to see what's going on. I will pick one that I know should have a value.

In [ ]:
df[df['incomplete_pass'].isnull()]

This shows me that there are just some games where there was just no recorded data other than the game_id and the teams that were supposed to play. These games contain only metadata and are therefor not useful. Since I know that at the very least, every game should have certain values, I can be sure that I'm not getting rid of any non-scoring game or any outlier game if I get rid of rows with NaN for many columns.

In [ ]:
## Drop a row if it has NaN in the following categories. While others are important, if a game has NaN for all of these values, it probably isn't worth recording.
df = df.dropna(subset=["pass_attempt","incomplete_pass","rush_attempt","tackled_for_loss","touchdown"], how='all')

In [ ]:
b = pd.Series(df.shape)
b

I am just checking to make sure that I am only getting rid of the same number of columns as I saw previously and no extras accidentally got lumped in.

In [ ]:
a-b

In [ ]:
#Check and create a new dataframe called dfpercents so I can easily split the remaining into two groups.
dfpercents = pd.DataFrame(df[df.columns[df.isnull().any()]].isnull().sum() * 100 / df.shape[0])
dfpercents.columns=['Percents']
dfpercents

In [ ]:
#low_mask=dfpercents[0]>=10 
high_mask = dfpercents.query('Percents >= 5')
low_mask = dfpercents.query('Percents < 5')

Success! Now we are only left with fields that either should have a value but an error caused one to be left out, or, a field that rarely occurs in a game, so that field (and fields directly related to it) are left with a null value for the majority of games. 



#Fill in missing values
For fields with a low number of null values, replacing them with the mean of the field is probably acceptable. For columns with a high number of NaN, it is probably best to replace with 0 to signify that the event did not occur in the game. 

The method I'm using for this particular dataset is probably not the most optimized, but I've found it works. Since there are 3 categories of data, clean, incorrect, and empty (change to zero), I want to split them up, apply transformations to each, then stitch them back together. 

I will use a "low mask" to select the names of all the columns that have a low percentage of NaN values and fill in those values with a mean for the column.

I will use a "high mask" to select the names of any column with a large amount of NaN values and change those to zeroes.

In [ ]:
low_columns = low_mask.reset_index()
low_columns = low_columns['index'].to_list()
low_columns

In [ ]:
#Now make a list of highs - same way basically.
high_columns = high_mask.reset_index()
high_columns = high_columns['index'].to_list()
high_columns

Now I just need to use these lists to replace the values. Then I should have no more NaN values without having to delete any more potentially useful data than absolutely necessary.

In [ ]:
df

In [ ]:
x = pd.Series(high_columns)
#print(x)
y = pd.Series(low_columns)
#print(y)
dfx = pd.DataFrame(df[x].fillna(value = 0))
# Check if it worked
dfx




In [ ]:
# Now I need to find the mean of JUST the columns I mean to change. I do this because finding the means of all 250 columns takes a long time, but finding just ~50 columns is much quicker.
mean = df[y].mean()
dfy = pd.DataFrame(df[y].fillna(value = mean))

In [ ]:
dfy

In [ ]:
#Join the two fixed dataframes into one for later integration with the original dataframe.
newdf = pd.concat([dfx, dfy], axis=1)
newdf

In [ ]:
df.update(newdf) #Use Pandas' update function to compare incoming data with old data and overwrite old values. This should leave the original data with no NaN values alone.

#Finished Data Set
While I'm 100% sure a lot of this can now be optimized and cleaned up, I wanted to leave it like this to show my train of thought. I am constantly checking to see what happened when I ran my code, I like to continue to compare previous versions to what I've done, and I like to take things step by step. It may take a little bit longer, but it reduces errors and enables me to get a better feel for the data.

Some of the columns in this dataset can probably be tweaked better than a simple mean, but as I'm time limited, I can't go through them column by column to determine the best course of action for each type of data. For example, I could hunt for outliers by implementing a mean() and max()/min() comparison and checking to see if any values were over a certain threshold above or below the mean (touchdowns >= 30 or something). I still think the results came out the way I wanted, so I'm happy with this for now. 

In [ ]:
df

In [ ]:
#Final Check
df[df.columns[df.isna().any()]].isna().sum() * 100 / df.shape[0]

Somewhere, somehow, very small percentages are still Null values. This will take deep digging. Or, I can just apply fillna() to the entire df and be done with it!